In [1]:
import pandas as pd
import transformers
import math

df = pd.read_csv('../../../data/yt_data.csv')
df.head()

/Users/lucyfarnik/miniforge3/envs/y3_ads/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_99195/815062686.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../../data/yt_data.csv')


,id,text,username,date,country,likes,n_children,title,platform,meta
0,Ugw9_y5-TI9gK34R_7Z4AaABAg,나도 사용해 봤는데 초기단계라서 그런지 오답을 말해주고 틀린 정보를 말해주는 경우도...,짱부자,2023-03-28T18:41:27Z,NaN,0.0,1.0,GPT4 출시! 만능 AI와 글/사진/영상으로 대화하기. 뜨거운 AI 경쟁 비하인드...,youtube,"{'vid_id': 'fIWpPnMW_0A', 'user_id': 'UCctowlf..."
1,UgxbDHn-wWEinu1lVDZ4AaABAg,감사합니다.,둥근달,2023-03-28T16:15:19Z,NaN,0.0,0.0,GPT4 출시! 만능 AI와 글/사진/영상으로 대화하기. 뜨거운 AI 경쟁 비하인드...,youtube,"{'vid_id': 'fIWpPnMW_0A', 'user_id': 'UCaFIABm..."
2,UgwA0C3c6Wrj80iqCpR4AaABAg,얼굴이쁜 피부미인이 설명하시니 듣기좋구 보기좋아요 넘 감사해요 주식투자로 중요한정...,둥근달,2023-03-28T16:14:56Z,NaN,0.0,0.0,GPT4 출시! 만능 AI와 글/사진/영상으로 대화하기. 뜨거운 AI 경쟁 비하인드...,youtube,"{'vid_id': 'fIWpPnMW_0A', 'user_id': 'UCaFIABm..."
3,UgzT6FJSnUMjorbg_dF4AaABAg,"영상 감사드립니다. 4버전 유료 결재해서 사용하고 있어요. 타로업을 하고 있는데, ...",청포도타로,2023-03-28T14:15:02Z,NaN,0.0,0.0,GPT4 출시! 만능 AI와 글/사진/영상으로 대화하기. 뜨거운 AI 경쟁 비하인드...,youtube,"{'vid_id': 'fIWpPnMW_0A', 'user_id': 'UCQnTg3v..."
4,UgwO6RKpWu0HGGMpOdl4AaABAg,Al 비하인드 스토리 진지한내용\n너무 놀랍고 대단합니다\n내미님 최신정보\n감...,패션줌마,2023-03-28T13:30:51Z,NaN,1.0,0.0,GPT4 출시! 만능 AI와 글/사진/영상으로 대화하기. 뜨거운 AI 경쟁 비하인드...,youtube,"{'vid_id': 'fIWpPnMW_0A', 'user_id': 'UC31322R..."


In [2]:
f"{df.shape[0]:,} comments in total"

'338,515 comments in total'

In [3]:
f"{100 * df['country'].notnull().sum() / df.shape[0]:.1f}% have country locations"

'6.1% have country locations'

In [4]:
f"There's {len(set(df[df['country'].notnull()]['country'].to_list()))} different countries"

"There's 156 different countries"

In [11]:
# add lang column
df['lang'] = None

In [16]:
lang_cls = transformers.pipeline('text-classification',
                                 'papluca/xlm-roberta-base-language-detection')
batch_size = 20
file_path = 'data/yt_w_langs.csv'
for batch_i in range(math.ceil(df.shape[0] / batch_size)):
  batch = df[batch_size*batch_i : batch_size*(batch_i+1)]
  langs = lang_cls(batch['text'].to_list())
  for i, lang in enumerate(langs):
    batch.at[i, 'lang'] = lang['label']
  if batch_i > 0 or os.path.exists(file_path): # if file exists, append
    batch.to_csv(file_path, mode='a', header=False, index=False)
  else: batch.to_csv(file_path, index=False)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).